In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [3]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)#参照ppt，实现policy gradient的loss function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        # s = cBandit.getBandit()
        # Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)    #利用e实现e-greedy
        
        reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print ("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...and it was right!")
else:
    print ("...and it was wrong!")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Running reward for the 4 bandits: [1. 0. 0. 0.]
Running reward for the 4 bandits: [ 0.  0.  1. 42.]
Running reward for the 4 bandits: [ 1. -1.  0. 85.]
Running reward for the 4 bandits: [  0.  -1.   0. 132.]
Running reward for the 4 bandits: [  3.   1.  -1. 176.]
Running reward for the 4 bandits: [  4.   0.  -1. 222.]
Running reward for the 4 bandits: [  4.   0.  -1. 270.]
Running reward for the 4 bandits: [  4.  -1.  -1. 315.]
Running reward for the 4 bandits: [  4.  -1.  -1. 363.]
Running reward for the 4 bandits: [  3.  -1.   1. 408.]
Running reward for the 4 bandits: [  3.  -1.   1. 458.]
Running reward for the 4 bandits: [  2.  -1.   2. 504.]
Running reward for the 4 bandits: [  2.   1.   2. 552.]
Running reward for the 4 bandits: [  0.  -1.   2. 598.]
Running reward for the 4 bandits: [  1.  -1.   1. 644.]
Running reward for the 4 bandits: [  0.  -2.   2. 691.]
Running reward for the 4 bandits: [ -1.  -1.  